# 2023-2 언어데이터과학 24강 (2023-11-29) 실습 (1) `gensim` 패키지와 Word2Vec 모델 훈련

## 오늘의 목표

1. 모두의 말뭉치 '온라인 대화 말뭉치'의 발화 자료로 Word2Vec Skip-gram 모델을 훈련시킬 수 있다.
2. 단어 사이의 의미 유사성을 벡터들의 코사인 유사도로 계량화하여 설명할 수 있다.
3. t-SNE 기법을 사용하여 단어 벡터들을 2차원 평면에 시각화할 수 있다.
4. Word2Vec 모델을 사용하여 연령대에 따른 단어의 분포 변화를 추적할 수 있다.

## 0. 준비

### 파이썬 모듈 설치하기

Python에서 Word2Vec 모델을 사용하기 위해서는 `gensim` 모듈이 필요하다.

In [1]:
%pip install gensim

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



     ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
     ---------------------------------------- 0.1/24.0 MB 3.5 MB/s eta 0:00:07
      --------------------------------------- 0.4/24.0 MB 3.7 MB/s eta 0:00:07
      --------------------------------------- 0.6/24.0 MB 4.0 MB/s eta 0:00:06
     - -------------------------------------- 0.7/24.0 MB 3.5 MB/s eta 0:00:07
     - -------------------------------------- 0.7/24.0 MB 2.9 MB/s eta 0:00:08
     - -------------------------------------- 0.8/24.0 MB 3.0 MB/s eta 0:00:08
     - -------------------------------------- 1.0/24.0 MB 2.9 MB/s eta 0:00:09
     - -------------------------------------- 1.1/24.0 MB 2.9 MB/s eta 0:00:08
     - -------------------------------------- 1.2/24.0 MB 2.7 MB/s eta 0:00:09
     -- ------------------------------------- 1.4/24.0 MB 2.9 MB/s eta 0:00:08
     -- ------------------------------------- 1.6/24.0 MB 3.0 MB/s eta 0:00:08
     -- ------------------------------------- 1.7/24.0 MB 

## 1. 데이터 가공

12–15강에서 만든 모두의 말뭉치 [온라인 대화 말뭉치] 파일을 읽고 `gensim` 모듈에서 사용 가능한 코퍼스로 가공하자.

In [2]:
import pandas as pd
from gensim.models import Word2Vec
from tqdm import tqdm

### 데이터 파일 읽기

In [3]:
DATA_PATH = '../data/NIKL_OM_form_age_sex.csv.tar.gz'

In [4]:
utterances = pd.read_csv(DATA_PATH, compression='gzip', on_bad_lines='skip')
utterances

,data/NIKL_OM_form_age_sex.csv,form,speaker_id,age,sex
0,MDRW2100000001.1.1,안녕하세요,MDRW2100000001_1,20대,여성
1,MDRW2100000001.1.4,이거 해봐요><,MDRW2100000001_1,20대,여성
2,MDRW2100000001.1.7,오 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,MDRW2100000001_1,20대,여성
3,MDRW2100000001.1.8,안챙겨도 잘커요,MDRW2100000001_1,20대,여성
4,MDRW2100000001.1.9,너무 맞는데요ㅜㅜ?,MDRW2100000001_1,20대,여성
...,...,...,...,...,...
2977836,MMRW2100000241.1.2775,한 번도 안 써봄...?,MMRW2100000241_2,20대,여성
2977837,MMRW2100000241.1.2776,그거 개꿀인디,MMRW2100000241_2,20대,여성
2977838,MMRW2100000241.1.2780,ㅋㅋㅋㅋㅋㅋ잠수복 개귀여웤ㅋㅋㅋㅋ,MMRW2100000241_2,20대,여성
2977839,MMRW2100000241.1.2786,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ가즈아!,MMRW2100000241_2,20대,여성


In [5]:
utterances = pd.read_csv(DATA_PATH, compression='gzip', on_bad_lines='skip')
utterances.dropna(inplace=True)
utterances.rename(columns={utterances.columns[0]: 'id'}, inplace=True)
utterances.set_index('id', inplace=True)
utterances

,form,speaker_id,age,sex
id,,,,
MDRW2100000001.1.1,안녕하세요,MDRW2100000001_1,20대,여성
MDRW2100000001.1.4,이거 해봐요><,MDRW2100000001_1,20대,여성
MDRW2100000001.1.7,오 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,MDRW2100000001_1,20대,여성
MDRW2100000001.1.8,안챙겨도 잘커요,MDRW2100000001_1,20대,여성
MDRW2100000001.1.9,너무 맞는데요ㅜㅜ?,MDRW2100000001_1,20대,여성
...,...,...,...,...
MMRW2100000241.1.2774,그 낚시대회 전용 투망 있을걸???,MMRW2100000241_2,20대,여성
MMRW2100000241.1.2775,한 번도 안 써봄...?,MMRW2100000241_2,20대,여성
MMRW2100000241.1.2776,그거 개꿀인디,MMRW2100000241_2,20대,여성


### 데이터 형변환

일반적으로 Python의 여러 라이브러리에서 코퍼스를 다룰 때는 한 문장을 단어들의 리스트로 표현하고, 코퍼스 전체를 문장들의 리스트로 표현한다.

지금 가지고 있는 데이터프레임에서는 문장에 해당하는 발화가 `str` 자료형이므로, `str.split()` 메소드를 사용하여 단어들의 리스트로 만들어 주자.

In [14]:
corpus = utterances['form'].apply(str.split) # edit this line
print(corpus[:5])

id
MDRW2100000001.1.1            [안녕하세요]
MDRW2100000001.1.4        [이거, 해봐요><]
MDRW2100000001.1.7    [오, ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ]
MDRW2100000001.1.8        [안챙겨도, 잘커요]
MDRW2100000001.1.9      [너무, 맞는데요ㅜㅜ?]
Name: form, dtype: object


]'/01112.3
31## 2. Word2Vec 모델 훈련

2
41"### 모델 초기화

In [15]:
model = Word2Vec(
    sg=1, #skit-gram
    min_count=5,#코퍼스에서 5번 미만으로 나온 단어는 생략 +
    vector_size=100, #vector dimension, 벡터 사이즈가 많을수록 단어 하나를 많은 수의 실수값으로 나타내는 것이기에 더 정교해짐, 하지만 시간과 저장 공간이 늘어나 
    window=2, # 앞의 2 단어 까지만 보겠음!
    negative=5,
)

### Vocabulary 구축

In [17]:
model.build_vocab(corpus_iterable=tqdm(corpus))

  0%|          | 0/2977840 [00:00<?, ?it/s]

100%|██████████| 2977840/2977840 [00:05<00:00, 546604.37it/s]


### 모델 훈련시키기

In [18]:
model.train(
    corpus_iterable=corpus,
    total_examples=model.corpus_count, # the number of sentences
    epochs=5 #코퍼스를 다섯 번 읽어라
)
# (trained_word_count, raw_word_count)

(32845577, 46433495)

### 훈련 결과

Vocabulary 확인

In [19]:
# 빈도순 상위 30개 단어
print(model.wv.index_to_key[:30])

['아', 'ㅋㅋ', 'ㅋㅋㅋ', '저는', '진짜', '근데', '너무', 'ㅎㅎ', '다', '저도', '좀', 'ㅋㅋㅋㅋ', '잘', '그', '나', '네', '더', '많이', '전', '그냥', '오', '난', 'ㅠㅠ', '맞아요', 'ㅋㅋㅋㅋㅋ', '안', '오늘', '저', '그래서', '그럼']


단어 벡터 확인

In [21]:
print(model.wv['ㅋㅋ'])

[ 0.3445659  -0.1511654  -0.3931686   0.40945736 -0.189487   -0.02498705
  0.30407605  0.546199   -0.6094101  -0.3205824   0.4679505   0.01710508
 -0.35031202  0.254841   -0.05741287 -0.19824481  0.37229434  0.49886516
 -0.04543076 -0.3166684   0.33684462  0.21715014  0.4798722  -0.5037191
  0.24849091 -0.5979235   0.3370465   0.06016716 -0.65940154  0.50831425
  0.33627406  0.3257374   0.58521193 -0.10557626 -0.14226006 -0.35132656
  0.4375407   0.17589377  0.10414356 -0.77771956  0.07782618 -0.49046075
  0.10020243 -0.6216116   0.19903535 -0.34703094 -0.45625192 -0.6094768
  0.5068315   0.35672563  0.47719055 -0.35134846  0.15068743 -0.160676
 -0.3860898   0.6720649   0.5068741   0.864657   -0.33997396  0.09905131
 -0.20693333 -0.68712467  0.3059252   0.03714753 -0.74384063  0.02604453
 -0.00557325 -0.01005473 -1.0203189   0.5830433   0.3362043  -0.32183525
  0.20322537  0.70713735 -0.52750194  0.54363453  0.0461545  -0.20336886
 -0.7957839  -0.01639933  0.12349543  0.10558452  0.546

## 3. Word2Vec 모델 활용

### `gensim`의 주요 기능



코사인 유사도 계산

In [22]:
print(model.wv.similarity(w1='펜', w2='연필'))
print(model.wv.similarity(w1='펜', w2='좋아요'))

0.90536475
0.38794887


평행사변형 모형

In [25]:
# 아빠 : 엄마 = X : 할머니
# 아빠 - 엄마 = X - 할머니
# X = 할머니 + 아빠 - 엄마
print(model.wv.most_similar(positive=['할머니', '아빠'], negative=['엄마'])) # ADD THIS LINE

[('할아버지', 0.8393165469169617), ('초등학교', 0.8178285956382751), ('name10', 0.8162828087806702), ('이모가', 0.8138353228569031), ('너희', 0.8085528016090393), ('name12', 0.8033748269081116), ('name11', 0.800075113773346), ('니네', 0.7999460697174072), ('name9', 0.7989993691444397), ('형부', 0.7969768047332764)]


우리 중에 스파이가 있는 것 같아

In [26]:
print(model.wv.doesnt_match(['ㅋㅋ', 'ㅎㅎ', '^^', 'ㅠㅠ']))

ㅠㅠ


### '완전'과 '아주'의 이웃 비교

In [27]:
model.wv.most_similar(['완전'])

[('짱', 0.7086948156356812),
 ('되게', 0.6537210941314697),
 ('대박', 0.6253469586372375),
 ('넘나', 0.6242092251777649),
 ('무지', 0.6102795600891113),
 ('디게', 0.6088290810585022),
 ('겁나', 0.6073566675186157),
 ('왕', 0.6060531139373779),
 ('아주', 0.5990931391716003),
 ('진짜', 0.593711793422699)]

In [29]:
model.wv.most_similar(['아주'])

[('매우', 0.7686690092086792),
 ('굉장히', 0.667973518371582),
 ('무척', 0.6672804951667786),
 ('무지', 0.6570212244987488),
 ('상당히', 0.6478555798530579),
 ('되게', 0.6371939182281494),
 ('정말', 0.6210654973983765),
 ('넘나', 0.6183302402496338),
 ('디게', 0.6104277968406677),
 ('완전', 0.5990931987762451)]

### 모델 저장하기

In [37]:
import os
os.makedirs('../models')

model.save('../models/word2vec-modu-online')